# Binary Planet Suppression: Toy Model

In [ ]:
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

mpl.rc('lines',linewidth = 1.5)
mpl.rc('font',size = 12)
mpl.rc('axes',labelsize = 14, linewidth=1.25)
mpl.rc('xtick',labelsize = 14)
mpl.rc('ytick',labelsize = 14)
# enable math fonts
mpl.rc('mathtext', default = 'regular')

import pickle

import pandas as pd

### Data from Sullivan, Gaia/Bedell, KOIs, KIC

In [ ]:
# read in Kendall's catalog, xmatched to all the stuff from Kepler that I need
with open('kendall+kepler.pkl', 'rb') as file:
       db = pickle.load(file)

# read in my kois table
with open('kic.pkl', 'rb') as file:
        kic = pd.read_pickle('kic.pkl')

# remove the studied binaries from the all_KOI catalog
# also remove false positives
# and non-positive parallaxes
kois_less = kois[ (~kois['kepoi_name'].isin(db['kepoi_name'])) & 
                 (kois['koi_disposition']!='FALSE POSITIVE') & 
                 (kois['parallax']>0)]

# Toy Model

next up I take the kois_less (kois with good params minus any that are binaries in kendall's list) and pretend every single star is actually a binary star with a separation drawn randomly from Kendall's sample's distribution of separations. Suppress the formation of > 2 Rearth planets using the suppression model and remake the histogram. Does it work?

In [ ]:
def suppression_factor(a_values):
    a_inner_true = 10  # AU (suppression 100%)
    a_outer_true = 200  # AU (suppression 0%)
    results = (np.log10(a_values) - np.log10(a_inner_true)) / (np.log10(a_outer_true) - np.log10(a_inner_true))
    print(results)
    return np.clip(results, a_min=0, a_max=1)

In [ ]:
# Number of stars in single KOIs catalog
n_stars = len(kois_less)

# Randomly draw separations from db[close] for each star
random_separations = np.random.choice(db[close]['SepAU'], size=n_stars, replace=True)

#plt.hist(random_separations, density=True)
#plt.hist(db['SepAU'], density=True)

# Suppress planet formation
my_factor = suppression_factor(random_separations)
planet_exists = np.zeros(n_stars, dtype=bool) 

# Condition 1: If Rp < 1.8, planet formation is not suppressed
planet_exists[kois_less['koi_prad'] < 1.8] = True

# Condition 2: If Rp >= 1.8, planet formation probabilistically suppressed
mask = (kois_less['koi_prad'] >= 1.8)

random_vals = np.random.rand(n_stars)  # uniform random [0,1) to compare to my_factor
planet_exists[mask] = random_vals[mask] < my_factor[mask]

realization = kois_less.loc[planet_exists == 1, 'koi_prad'].to_numpy()

In [ ]:
plt.hist(random_separations[mask])
plt.hist(random_separations[mask][planet_exists[mask]])

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6,5), sharex=True, sharey=True)


# planets with good radii in binary systems with rho<100 au and planet period < 100 days
axes[0].hist(db['Rppri'][good & close], range=[0.2,4], bins=20, 
         density=True, histtype='step',
        label='Close binaries')

# compared to the kois that are at least not definite binaries with good radii and per<100 days
axes[0].hist(kois_less['koi_prad'][kgood], range=[0.2,4], bins=20, 
         density=True, histtype='step',
        label='Single stars')

axes[0].set_ylabel('Density')
axes[0].legend()


# planets with good radii in binary systems with rho<100 au and planet period < 100 days
axes[1].hist(db['Rppri'][good & close], range=[0.2,4], bins=20, 
         density=True, histtype='step',
        label='Close binaries')

# compared to the kois that exist in simulation still
axes[1].hist(kois_less['koi_prad'][kgood & planet_exists], range=[0.2,4], bins=20, 
         density=True, histtype='step',
        label='Suppressed')

plt.legend()
plt.xlabel('Rp')
plt.ylabel('Density')
plt.show()

In [ ]:
"KIC"+kois['kepid'][np.isnan(kois['parallax'])].astype('str')

In [ ]:
kois.columns

In [ ]:
# might be duplicates in db do i want that